In [ ]:
from sklearn.metrics import mean_squared_error, confusion_matrix, auc

In [ ]:
def roc_params(metric, label, interp=True):
    fpr = []
    tpr = []
    thr = []
    thr_list = list(np.linspace(0, metric.max(),1001))

    fp = 1
    ind = 0
    while fp > 0:
        threshold = thr_list[ind]
        ind += 1

        y = (metric>threshold)
        tn, fp, fn, tp = confusion_matrix(label, y).ravel()

        fpr.append( fp/(tn + fp) )
        tpr.append( tp/(tp + fn) )
        thr.append( threshold )

    while tp > 0:
        threshold = thr_list[ind]
        ind += 1
        y = (metric>threshold)
        tn, fp, fn, tp = confusion_matrix(label, y).ravel()

    
    fpr = fpr[::-1]
    tpr = tpr[::-1]
    thr = thr[::-1]

    if interp:
        fpr_base = np.linspace(0, 1, 101)
        tpr = list(np.interp(fpr_base, fpr, tpr))
        thr = list(np.interp(fpr_base, fpr, thr))
        fpr = list(fpr_base)

    fpr.insert(0, 0)
    tpr.insert(0, 0)
    thr.insert(0, threshold)

    return tpr, fpr, thr

def compute_auc(tpr, fpr):
    auc = 0
    for i in range(1, len(fpr)):
        auc += (fpr[i] - fpr[i - 1]) * (tpr[i] + tpr[i - 1]) / 2
    return auc

# def detection(df_metrics, column_name='wse', threshold_min=1000, threshold_max=np.inf, selector='group',
#               detection_param='detection_sum', detection_param_threshold=None):
#     # df_relevant contains data from nodes that, at some point, have lower<=wse<=upper, and their neighbors.
#     # nodes are put into groups if they are close to each other.

#     if detection_param_threshold is None:
#         detection_param_threshold = df_metrics.timestamp.nunique()//2

#     df_relevant = mma.relevant_neighborhood(df_metrics, column_name=column_name,
#                                             lower=threshold_min, upper=threshold_max,
#                                             only_relevant=True, return_df=True, plot=False, filter_dates=False)

#     # Treating disconnected nodes as individual groups. Assining new values
#     new_group_values = df_relevant.query('group==0').pid.factorize()[0] + df_relevant.group.max()+1
#     df_relevant.loc[df_relevant.group==0, 'group'] = new_group_values


#     df_relevant['detection'] = (df_relevant[column_name]>=threshold_min) & (df_relevant[column_name]<=threshold_max)
#     df_detection = df_relevant.groupby('pid').agg({column_name:['max','mean'],
#                                                     'detection':['sum',mma.consecutive_ones],
#                                                     'group':'mean'}).reset_index()

#     df_detection.columns = [f"{level1}_{level2}" if level2 else level1 for level1, level2 in df_detection.columns]
#     df_detection.rename({'group_mean':'group'}, axis=1, inplace=True)

#     query = f'{detection_param}>{detection_param_threshold}'
#     selected = df_detection.query(query)[selector].unique()

#     return df_relevant, selected

# def skew(df):
#     return np.abs(sp.stats.skew(df.mean_velocity))


# def compute_metric(df_test, cut=2, radius=15):

#     df_metrics = []
#     for cluster in sorted(df_test.cluster.unique()):

#         df, nodes = mma.treat_nodes(df_test.query('cluster==@cluster'))
#         G, nodes['subgraph'] = mma.NNGraph(nodes, radius=radius, subgraphs=True)

#         df_metrics_cluster = []
#         for sub_index in sorted(nodes.subgraph.unique())[1:]:

#             subnodes = nodes.query('subgraph==@sub_index').copy()
#             subdf = df[df.pid.isin(subnodes.pid)].copy()

#             G = mma.NNGraph(subnodes, radius=radius)

#             w, V = np.linalg.eigh(G.L.toarray())
#             wh = np.ones(G.N)
#             wh[w<cut] = 0
#             Hh = V @ np.diag(wh) @ V.T

#             smoothed = subdf[['pid', 'timestamp', 'smoothed' ]].pivot(index='pid', columns='timestamp')

#             subdf['hf'] = np.abs((Hh @ smoothed.values).reshape((-1,), order='C'))

#             df_metrics_cluster.append(subdf)

#         df_metrics_cluster = pd.concat(df_metrics_cluster)
#         df_metrics.append(df_metrics_cluster)

#     df_metrics = pd.concat(df_metrics)
#     return df_metrics


# def hfilter(G, cut=2):
#     L = G.L.toarray()
#     w, V = np.linalg.eigh(L)
#     wh = np.ones(G.N)
#     wh[w<cut] = 0
#     Hh = V @ np.diag(wh) @ V.T
#     return Hh

# def matplotlib_roc(save=None, ax=None):
#     matplotlib.rcParams.update({'font.size': 20})
#     matplotlib.rcParams.update({'font.family': 'Times New Roman'})

#     if ax is None:
#         fig, ax = plt.subplots(figsize=(12,5))
#     # sc = ax.scatter(fpr, tpr, c=thr, cmap='viridis', label='Threshold')
#     sc = ax.plot(fpr, tpr, linestyle='dotted', linewidth=1, color='black')

#     # # Colorbar
#     # cbar = plt.colorbar(sc, ax=ax)
#     # cbar.set_label('Threshold', rotation=270, labelpad=15)

#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     # plt.grid()
#     # plt.tight_layout()

#     if save is not None:
#         plt.savefig(save, transparent=True)


In [ ]:
# import numpy as np

# def synth_graph(seed=0):
#     if seed is not None:
#         np.random.seed(seed)

#     # Define the length and position of x
#     minx = 0
#     maxx = 300
#     stepx = 5
#     posx = np.arange(minx, maxx, stepx)

#     # Define the length and position of y
#     miny = -15
#     maxy = +15
#     numy = 3
#     posy = np.linspace(miny, maxy, numy)

#     # Create a meshgrid of x and y
#     X, Y = np.meshgrid(posx, posy)

#     # Combine x and y into a single array of positions
#     pos = np.c_[X.ravel(),Y.ravel()]

#     # Add random noise to the positions
#     pos = pos + np.random.randn(pos.shape[0], pos.shape[1])

#     # Add extra random nodes to the positions
#     extra_nodes = np.random.uniform(low=[0, -15], high=[maxx, 15], size=(len(pos)//10, 2))
#     pos = np.r_[pos, extra_nodes]

#     # Sort the positions by x-coordinate
#     pos = pos[np.argsort(pos[:,0]),:]

#     # Create a graph using the positions
#     G = pygsp.graphs.NNGraph(pos,
#                             NNtype='radius',
#                             epsilon = 15, sigma = 100,
#                             center=False, rescale=False)

#     plotting_params = {'edge_color':'lightblue', 'edge_width':2,'vertex_color':'black', 'vertex_size':150}
#     G.plotting.update(plotting_params)
#     return G

# def ramp_to_plateou(pos, slope, start=-np.inf, end= np.inf):
#     mask =  (pos[:,0] >= start) * (pos[:,0]< end)
#     ramp = slope*(pos[:,0] - start)
#     ramp = ramp*mask
#     ramp[pos[:,0]>=end] = ramp.max()
#     return ramp

# def create_data(G, anomaly=0.1, size=20, noise_var = 1e-2, signal_power = 1, seed=None, max_slope=1, eigs=1):

#     np.random.seed(seed)

#     pos = G.coords

#     w, V = np.linalg.eigh(G.L.toarray())
#     w[eigs:] = 0 # Frequency filter

#     displacement = np.random.randn(G.N, size)
#     displacement = V @ np.diag(w) @ V.T @ displacement # filtering
#     # normalizing for desired power
#     displacement = np.sqrt(signal_power*G.N)*displacement/(np.linalg.norm(displacement,axis=0))

#     noise = np.sqrt(noise_var)*np.random.randn(G.N,size)

#     # terrain corresponds to a ramp to a plateou in the horizontal direction
#     slope = max_slope*np.random.rand() # makes small difference given proportional anomaly and scaler
#     start = pos[:,0].max()*np.random.rand()*0.5 # Slope always start in the first half
#     # end = start + (pos[:,0].max()-start)*np.random.rand()

#     min_slope_dist = 100
#     end = start + min_slope_dist + (pos[:,0].max()-start-min_slope_dist)*np.random.rand() #At least 50m of slope

#     terrain = ramp_to_plateou(pos, slope=slope, start=start, end=end).reshape((-1,1))
#     terrain = np.tile(terrain, (1,size)) # Matching number of samples (size)

#     ptp = terrain.ptp()

#     signal = displacement + noise + terrain
#     label = np.zeros(signal.shape)

#     for timestamp in range(size):

#         anomalous_sensors = np.vstack([
#                              np.random.choice(np.arange(0, G.N//3), size=(2,1), replace=False),
#                              np.random.choice(np.arange(G.N//3, 2*G.N//3), size=(2,1), replace=False),
#                              np.random.choice(np.arange(2*G.N//3, G.N), size=(2,1), replace=False)]
#                             ).flatten()
#         signal[anomalous_sensors, timestamp] += np.random.choice([anomaly*ptp, -anomaly*ptp], size=(6,))
#         label[anomalous_sensors, timestamp] = 1

#     return signal, label

In [ ]:
def generate_synthetic_graph(N, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    N_grid_points = int(np.floor(0.9*N))
    N_rand_points = int(np.ceil(0.1*N))

    connected = False

    while not connected:

        # Generating grid points

        # Initialize starting position
        current_position = [0, 0]

        # Store visited points
        visited_points = set()
        visited_points.add(tuple(current_position))

        # Generate N points
        while len(visited_points) < N_grid_points:
            # Update the current position by taking a vertical or horizontal step
            # horizontal can be +-5, vertical can be +-15

            if np.random.rand()>0.5:
                current_position[0] += np.random.choice([-5, 5])
            else:
                current_position[1] += np.random.choice([-14, 14])

            # Add the new position to the set of visited points
            visited_points.add(tuple(current_position))

        points_list = list(visited_points)

        # Generating random points
        reference_pos = np.random.choice(a=N_grid_points, size=N_rand_points, replace=False)
        reference_points = [list(points_list[i]) for i in reference_pos]

        for point in reference_points:
            point[0] += 10*np.random.random()
            point[1] += 10*np.random.random()
            visited_points.add(tuple(point))
        

        # Convert the set of visited points to a list
        pos = np.array(list(visited_points))
        pos = pos + 0.5*np.random.randn(pos.shape[0], pos.shape[1])

        G = pygsp.graphs.NNGraph(pos,
                                NNtype='radius',
                                epsilon = 15, sigma = 100,
                                center=False, rescale=False)
        connected = G.is_connected()

    return G

In [ ]:
def generate_ramp(G, size):
    # Auxiliary function for generating data.
    # Returns array with "size" random ramps on the given graph. A ramp emulates regions with different behaviors.
    # Direction of each ramp is randomly horizontal or vertical
    # Slope, start and end positions of the ramp are random, within some constraints.
    
    pos = G.coords
    min_slope = 0.1
    max_slope = 1
    
    # Initialize an empty matrix to store ramp vectors
    ramp_matrix = np.zeros((len(pos), size))

    for i in range(size):
        # Randomly choose the direction of the ramp (0, horizontal or 1, vertical)
        if np.random.rand() > 0.5:
            direction = 0
        else:
            direction = 1

        # Generate a random slope within the specified maximum slope
        slope = np.random.uniform(low=min_slope, high=max_slope)

        # Calculate the minimum ramp length as a fraction of the peak-to-peak range
        min_slope_dist = 0.25 * pos[:, direction].ptp()

        # Determine the starting and ending positions
        start = pos[:, direction].min() + pos[:, direction].ptp() * np.random.rand() * 0.5
        end = start + min_slope_dist + (pos[:, direction].max() - start - min_slope_dist) * np.random.rand()

        # Generate the ramp vector based on the slope and the position within the range
        ramp = slope * (pos[:, direction] - start)

        # Applying a mask that defines where the ramp exists
        mask = (pos[:, direction] >= start) * (pos[:, direction] < end)
        ramp = ramp * mask

        # Set values beyond the end of the ramp to the maximum ramp value
        ramp[pos[:, direction] >= end] = ramp.max()

        # Store the ramp vector in the matrix
        ramp_matrix[:, i] = ramp

    return ramp_matrix

def generate_smooth(G, size):
    # Auxiliary function for generating data
    # Returns array with "size" smooth signals by filtering white noise with the two first eigenvectors of the graph
    
    w, V = np.linalg.eigh(G.L.toarray())

    # Low-pass filter
    h = np.ones(len(w))
    h[0] = 1
    h[1] = 0.1
    h[2:] = 0 

    # Generating and filtering white noise to create a smooth graph signal
    displacement = np.random.randn(G.N, size) 
    displacement = V @ np.diag(h) @ V.T @ displacement

    # Normalizing signal: average power sum(x^2)/N = 1
    displacement = np.sqrt(G.N)*displacement/(np.linalg.norm(displacement,axis=0))

    return displacement


def pick_non_adjacent(G, n):
    # Auxiliary function for generating anomalies.
    # Picks n nodes that are not adjacent to each other.
    A = G.A.toarray()
    np.fill_diagonal(A,1)

    tries = 0

    while tries < 100000:
        tries+=1

        fail = 0
        possible_nodes= list(np.arange(G.N))
        picked_nodes = []

        for node in range(n):
            if len(possible_nodes) == 0:
                fail = 1
            else:
                pick = np.random.choice(possible_nodes)
                picked_nodes.append(pick)
                neighbors = np.where(A[pick,:])[0]
                possible_nodes = list( set(possible_nodes) - set(neighbors)  )

        if not fail:
            return picked_nodes
    
    print('Maximum tries reached. Reduce n')

def generate_anomaly(G, signal):
    # signal is expected to be a matrix containing different signals in each column
    # each column will receive different anomalies

    anomaly_factor_min = 0.05
    anomaly_factor_max = 0.15

    N = signal.shape[0]
    size = signal.shape[1]

    # Defining number of anomalous sensors per signal. Each column of signal has a different number of anomalies
    # This value is up to 5% the number of sensors (rounded up)
    percentage_per_signal = 0.05*np.random.rand(size) + 1e-5 # value added to avoid zero
    number_per_signal = np.ceil(percentage_per_signal*N)

    anomalous_positions = [pick_non_adjacent(G,int(n)) for n in number_per_signal]

    anomaly = np.zeros(signal.shape)
    for i in range(size):
        n = int(number_per_signal[i])
        anomalous_positions = pick_non_adjacent(G,n)

        anomaly[anomalous_positions, i] = signal[:,i].ptp()*np.random.uniform(low=anomaly_factor_min,
                                                                              high=anomaly_factor_max,
                                                                              size=n)*np.random.choice([+1,-1],size=n)
        
    label = np.zeros(signal.shape)
    label[np.where(anomaly)] = 1

    return anomaly, label

def generate_data(G, size):    

    # Generating healthy data
    displacement = generate_smooth(G, size)
    ramp = generate_ramp(G, size)
    noise = np.sqrt(0.1)*np.random.randn(G.N,size) #noise power = 0.1, SNR = 20 dB

    signal = displacement + ramp + noise

    # Introducing anomalies
    anomaly, label = generate_anomaly(G, signal)
    signal = signal + anomaly
    
    return signal, label